In [ ]:
#
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 22 14:48:16 2019

@author: Administrator
"""

import wx
import wx.grid
import os
from data_info import GetData
from GetAbstract import GetAbstract
from MyDataFrame import MyDataFrame
from ShowAbstract import ShowAbstract
from WordsCloud import WordsCloud
from ShowCloud import ShowCloud
from OpenPDF import OpenPDF

# 自定义窗口类MyFrame
class MyFrame(wx.Frame):
    column_names =['标题','类别','关键字' ]
    data=[['','','']]
    def __init__(self):
        super().__init__(parent=None, title='电子文献检索', size=(1000,700))
        self.Centre()  # 设置窗口居中
        self.panel = wx.Panel(self)
        #最底层box
        vbox0 = wx.BoxSizer(wx.VERTICAL)
        #上下两个box
        vbox1 = wx.BoxSizer()#水平
        vbox2 = wx.BoxSizer()#水平
        #1.在最底层添加标题       
        header_text = wx.StaticText(self.panel,label='电子文献检索系统',style=wx.TE_CENTER)
        font = wx.Font(18,wx.DEFAULT,wx.NORMAL,wx.NORMAL)
        header_text.SetFont(font)

        #2.在上层box添加内容
        openfiles_button = wx.Button(self.panel,id=1,label='打开')#打开文件夹
        openfiles_button.Bind(wx.EVT_BUTTON,self.OnOpen)       
        self.openfiles_text = wx.TextCtrl(self.panel,value='',style=wx.TE_LEFT)
        
        #搜索的文本框
        self.search_text = wx.TextCtrl(self.panel,value='',style=wx.TE_LEFT)
        search_button = wx.Button(self.panel,id=1,label='搜索')
        search_button.Bind(wx.EVT_BUTTON,self.SearchFiles)
     
        #3.向下层box添加内容   
        #下面box中的box
        vbox3 = wx.BoxSizer(wx.VERTICAL)#垂直
        vbox4 = wx.BoxSizer(wx.VERTICAL)       
        #网格
        
        self.grid = wx.grid.Grid(self.panel)
        self.grid.CreateGrid(len(self.data), len(self.data[0]))#行、列
        self.GridSetData(self.data,self.column_names)

        self.grid.SetColSize(0,330)
        self.grid.SetColSize(1,80)
        self.grid.SetColSize(2,240)
        
        # 设置行和列自定调整
        #self.grid.AutoSize()
        #绑定事件，当用户在行或列的标签区域敲击鼠标左键时触发该事件
        self.Bind(wx.grid.EVT_GRID_LABEL_LEFT_CLICK, self.OnLabelLeftClick)  
        self.Bind(wx.grid.EVT_GRID_LABEL_LEFT_DCLICK,self.OnLableLeftDClick)
        
        self.path_show = wx.StaticText(self.panel,label='当前文件夹路径：')
               
        #下面右侧控件
        #分类下拉列表
        hbox5 = wx.BoxSizer()
        fenlei_text = wx.StaticText(self.panel,label='分类：')
        list1 = ['A_All','Computer', 'Art', 'Literature','Education','Philosophy','History','Space','Energy','Electronics','Communication','Mine','Transport','Enviornment','Agriculture','Economy','Law','Medical','Military','Politics','Sports']
        #ch1下拉列表，value为默认值，choices对应一个列表，style=wx.CB_SORT使选择列表中的元素按字母顺序显示。
        fenlei = wx.Choice(self.panel, -1, choices=list1, style=wx.CB_SORT)
        self.Bind(wx.EVT_CHOICE, self.on_choice, fenlei)#绑定事件
        
        #按钮      
        ciyun_button = wx.Button(self.panel,label='词云')
        self.Bind(wx.EVT_BUTTON, self.show_cloud,ciyun_button)
        abstract_button = wx.Button(self.panel,label='摘要')
        self.Bind(wx.EVT_BUTTON, self.get_abstract,abstract_button)
        similar_button= wx.Button(self.panel,label='相似文档')
             
        #向上层box添加内容       
        #将打开文件夹添加到上层box
        vbox1.Add(openfiles_button,1,flag=wx.EXPAND)       
        vbox1.Add(self.openfiles_text,3,flag= wx.LEFT | wx.EXPAND,border=10)
        #将搜索框添加到上层box
        vbox1.Add(self.search_text,5,flag= wx.LEFT | wx.EXPAND,border=300)
        vbox1.Add(search_button,1,flag=wx.EXPAND | wx.LEFT, border=10)
        
        
        #想下层box添加内容
        vbox3.Add(self.path_show,1,flag=wx.EXPAND | wx.LEFT, border=10)
        vbox3.Add(self.grid,10,flag= wx.EXPAND)
        hbox5.Add(fenlei_text,1,flag=wx.EXPAND)
        hbox5.Add(fenlei,3,flag=wx.EXPAND)
        vbox4.Add(hbox5,flag= wx.EXPAND)
        vbox4.Add(ciyun_button,flag= wx.EXPAND | wx.TOP,border = 150)
        vbox4.Add(abstract_button,flag= wx.EXPAND | wx.TOP, border = 20)
        vbox4.Add(similar_button,flag= wx.EXPAND | wx.TOP, border = 20)
    
    
        #向下层box添加内容
        
        vbox2.Add(vbox3,5,flag= wx.EXPAND)
        vbox2.Add(vbox4,1,flag= wx.EXPAND | wx.ALL, border = 20)
        
     
        #向底层box添加内容       
        vbox0.Add(header_text,flag = wx.TOP | wx.EXPAND | wx.CENTER,border = 20) #将标题添加到底层box
        vbox0.Add(vbox1,1,flag=wx.ALL | wx.EXPAND,border=20)
        vbox0.Add(vbox2,9,flag=wx.ALL | wx.EXPAND,border=20)
        self.panel.SetSizer(vbox0)
        
    
    def GridSetData(self,data,column_names):
        #添加数据
        for row in range(len(data)):
            for col in range(len(data[row])):
                self.grid.SetColLabelValue(col, column_names[col])#设置列标题
                self.grid.SetCellValue(row, col, data[row][col])#设置单元格内容
                
                #SetRowLabelValue()，SetColLabelValue()和SetCellValue()，它们实际上设置显示在网格中的值
                #SetCellValue()方法要求一个行索引、一个列索引和一个值。而其它两个方法要求一个索引和一个值。
              
    def OnOpen(self,event):
        dlg = wx.DirDialog(None,message='对话框标题',style=wx.DD_DEFAULT_STYLE)
        
        if dlg.ShowModal() == wx.ID_OK:
            self.file = dlg.GetPath()          
            #print(self.file)#file为文件夹路径名
            self.openfiles_text.SetLabelText(self.file)# 在下方写入路径
            self.path_show.SetLabelText('当前文件夹路径：'+self.file)
            dlg.Destroy()

        #向后台传入路径file，获取data
        gd = GetData()
        self.data= gd.getInitData()
        #print(self.data)
        #print(len(self.data))
        
        self.grid.AppendRows(numRows = len(self.data) - 1)
        self.GridSetData(self.data,self.column_names)
        # 设置行和列自定调整
        #self.grid.AutoSize()
  
            
    def SearchFiles(self,event):
        str_search = self.search_text.GetValue()
        print(str_search)
        #print('搜索文件')
        gd = GetData()
        simfiles = gd.getSearchFiles(str_search)
        print(len(self.data)) #1.50
        print(len(simfiles))# computer 5
        print(simfiles)       
        if len(simfiles) == 0:
            nulldata = [['','','']]
            if len(self.data) == 1:
                self.data = nulldata
            else:               
                numrows = len(self.data) - 1 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
                self.data = nulldata
        else:
            if len(self.data) > len(simfiles):    
                numrows = len(self.data) - len(simfiles) 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
            elif len(self.data) < len(simfiles):
                numrows = len(simfiles) - len(self.data)
                self.grid.AppendRows(numRows = numrows)
            else:
                pass
            self.data = simfiles
        self.GridSetData(self.data,self.column_names)
        
      
    #网格点击事件
    def OnLabelLeftClick(self, event):
        print("RowIdx：{0}".format(event.GetRow()))#行索引
        print("ColIdx：{0}".format(event.GetCol()))#列索引
        #print(self.data[event.GetRow()])
        self.fileselect = self.data[event.GetRow()]
        print(self.fileselect)
        event.Skip()#确保还可以响应其他事件处理
 
    
    def OnLableLeftDClick(self,event):
        #print("RowIdx：{0}".format(event.GetRow()))#行索引
        #print("ColIdx：{0}".format(event.GetCol()))#列索引
        #print(self.data[event.GetRow()])
        self.fileselect = self.data[event.GetRow()]
        
        print(self.file) #文件夹路径
        #self.fileselect[0] 文件名
        
        allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
        for i in range(len(allfiles)):
            if allfiles[i].find(self.fileselect[0]) != -1:
                filepath = allfiles[i]
                print(filepath)
                op = OpenPDF()
                op.experimentGuide(1,filepath)
                #os.popen(r'"C:\Program Files (x86)\Foxit Software\Foxit Reader\FoxitReader.exe" filepath ')
    

    def get_abstract(self,event):
        try:
            filename = self.fileselect[0]
            print(filename)
            mdf = MyDataFrame()
            df = mdf.new_DataFrame()
            for i in range(len(df)):
                if df.filename[i] == filename:
                    file_content = df.content[i]

            #获取摘要
            ga = GetAbstract()
            abstract_contents = ga.readAbstract(filename)
            if len(abstract_contents) == 0:
                abstract_contents = ga.generateAbstract(file_content)
            else:
                pass
            print(abstract_contents)
            
            dialog = ShowAbstract(abstract_contents)
            dialog.Show()
            
        except:
            pass
            

        #遍历文件夹及其子文件夹
    def list_all_files(self,rootdir):
        _files = []
        list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.list_all_files(path))
            if os.path.isfile(path):
                _files.append(rootdir +'\\'+list[i])
        return _files
            
            
        
    def show_cloud(self,event):
        try:
            filename = self.fileselect[0]
            print(filename)
            wc = WordsCloud()
            wc.saveCloud(filename)
            sc = ShowCloud(filename)
            sc.Show()
        except:
            pass


    #分类按钮下拉选择
    def on_choice(self, event):
        print('选择 {0}'.format(event.GetString()))
        catagory_select = event.GetString()
        gd = GetData()
        catagory_data = gd.getCatagoryData(catagory_select)
        print(len(self.data)) #1.50
        print(len(catagory_data))# computer 5
        #print(self.data)
        print(catagory_data)
        if len(catagory_data) == 0:
            nulldata = [['','','']]
            if len(self.data) == 1:
                self.data = nulldata
            else:               
                numrows = len(self.data) - 1 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
                self.data = nulldata
        else:
            if len(self.data) > len(catagory_data):    
                numrows = len(self.data) - len(catagory_data) 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
            elif len(self.data) < len(catagory_data):
                numrows = len(catagory_data) - len(self.data)
                self.grid.AppendRows(numRows = numrows)
            else:
                pass
            self.data = catagory_data
        self.GridSetData(self.data,self.column_names)
        

class App(wx.App):

    def OnInit(self):
        # 创建窗口对象
        frame = MyFrame()
        frame.Show()
        return True


if __name__ == '__main__':
    app = App()
    app.MainLoop()  # 进入主事件循环

    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.258 seconds.
Prefix dict has been built succesfully.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
选择 Art
50
0
[]
选择 Computer
1
5
[['基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', 'Computer', '聚类,算法,数据挖掘,ik,档案'], ['基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', 'Computer', '胴体,模型,图像,算法,样本'], ['数据挖掘技术在语音识别中的应用_许元洪_郭琼', 'Computer', '语音,样本,算法,说话人识别,降维'], ['数据挖掘技术在风力发电中的应用综述_曾文珺', 'Computer', '数据挖掘,算法,模型,故障诊断,机组'], ['浅析机械设计制造及其自动化中计算机技术的应用_郑宏栗', 'Computer', '机械设计,自动化,制造,计算机技术,发展']]
选择 A_All
5
50
[['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', 'Enviornment', '合格率,消毒,医疗机构,灭菌,监测'], ['_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', 'Economy', '教学,国贸,国际,专业,投资'], ['三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', 'Medical', '互认,医疗,检查,三级,医院'], ['不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', 'Medical', '检查,互认,医院,医疗,等级'], ['乡村振兴战略下环巢湖休闲农业发展研究_沈东生', 'Agriculture', '巢湖,休闲,农业,旅游,产品'], ['二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', 'Medical', '互认,检查,影像学,医院,医疗'], ['关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', 'Sports', '建筑工程,管理,项目,

# 网格删除行

In [ ]:
#
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 22 14:48:16 2019

@author: Administrator
"""

import wx
import wx.grid
from data_info import GetData


# 自定义窗口类MyFrame
class MyFrame(wx.Frame):
    column_names =['标题','类别','关键字' ]
    data=[['111','111','111'],['111','111','111'],['111','111','111'],['111','111','111'],['111','111','111'],['111','111','111']]
    def __init__(self):
        super().__init__(parent=None, title='电子文献检索', size=(1000,700))
        self.Centre()  # 设置窗口居中
        self.panel = wx.Panel(self)
        vbox = wx.BoxSizer()
        self.grid = wx.grid.Grid(self.panel)
        self.grid.CreateGrid(len(self.data), len(self.data[0]))#行、列
        self.GridSetData(self.data,self.column_names)

        self.grid.SetColSize(0,330)
        self.grid.SetColSize(1,80)
        self.grid.SetColSize(2,240)
        
        # 设置行和列自定调整
        #self.grid.AutoSize()
        #绑定事件，当用户在行或列的标签区域敲击鼠标左键时触发该事件
        
        search_button = wx.Button(self.panel,id=1,label='搜索')
        search_button.Bind(wx.EVT_BUTTON,self.SearchFiles)
        
        
        vbox.Add(self.grid,10)
        vbox.Add(search_button,1)
        self.panel.SetSizer(vbox)
        
    
    def GridSetData(self,data,column_names):
        #添加数据
        for row in range(len(data)):
            for col in range(len(data[row])):
                self.grid.SetColLabelValue(col, column_names[col])#设置列标题
                self.grid.SetCellValue(row, col, data[row][col])#设置单元格内容
                
                #SetRowLabelValue()，SetColLabelValue()和SetCellValue()，它们实际上设置显示在网格中的值
                #SetCellValue()方法要求一个行索引、一个列索引和一个值。而其它两个方法要求一个索引和一个值。
              
    
  
            
    
    #分类按钮下拉选择
    def SearchFiles(self, event):
        print('-------------------------')
        #data 6行 
        newdata = [['111','111','111'],['111','111','111']]
        numrows = len(self.data) - len(newdata) 
        self.grid.DeleteRows(pos = 1,numRows = numrows)
    
        self.data = newdata
        self.GridSetData(self.data,self.column_names)
        

class App(wx.App):

    def OnInit(self):
        # 创建窗口对象
        frame = MyFrame()
        frame.Show()
        return True


if __name__ == '__main__':
    app = App()
    app.MainLoop()  # 进入主事件循环

    
    
#注意initdata重复读入

# 弹出另一个窗口

In [ ]:

 
import wx
#自定义窗口类MyFrame
class MyFrame(wx.Frame):
    def __init__(self):
        super().__init__(parent=None,title="一对一事件处理",size=(400,300))
        #自定义frame，然后让它继承wxFrame，并提供构造方法
        self.Centre()#设置窗口居中
        
        #在frame中放一个面板，把其他控件和窗口放到面板中去
        panel = wx.Panel(parent=self)
        
        #把面板添加到某一个容器/窗口中，就让parent指向谁，parent为父容器/父窗口，self为当前的frame
        #statictext成员变量，因为要在onclick中访问
        self.statictext = wx.StaticText(parent=panel,pos=(110,20))
        #放一个静态文本框，相当于label，放到panel中，所以parent=panel，pos为位置
        
        b=wx.Button(parent=panel,label='OK',pos=(100,50))
        self.Bind(wx.EVT_BUTTON,self.on_click,b)#当前对象调用绑定方法
        
        
    def on_click(self,event):
        print(event)
        self.statictext.SetLabelText('hello world')
        
        Dialog = SketchAbout()
        Dialog.Show()
        
class SketchAbout(wx.Dialog):
    def __init__(self):
        print('------------------')
        wx.Dialog.__init__(self,None,-1,'摘要',size=(400,300))
        self.Centre()#设置窗口居中
        panel = wx.Panel(parent=self)
        
        #创建垂直方向box布局管理器
        vbox=wx.BoxSizer(wx.VERTICAL)
        
        self.statictext = wx.StaticText(panel,label='statictext1',style=wx.ALIGN_CENTER_HORIZONTAL)

        b1= wx.Button(panel,1,label='确定')
        self.Bind(wx.EVT_BUTTON,self.closewindow ,b1)
        vbox.Add(self.statictext,6,flag = wx.ALIGN_CENTER | wx.EXPAND)
        vbox.Add(b1,1,flag=wx.ALIGN_RIGHT)
        panel.SetSizer(vbox)
          
    def closewindow(self,event):
        self.Close(True)
        
#应用程序对象自定义，可以捕获应用程序生命周期
class App(wx.App):
    def OnInit(self):
        #创建窗口对象
        frame = MyFrame()
        frame.Show()
        return True
    
    def OnExit(self):
        print('应用程序退出')
        return 0
    
#当前模块名是不是主模块，即应用程序的入口  
if __name__ == '__main__':
    app = App()#调用上面函数
    app.MainLoop()#进入主事件


------------------
------------------


# 打开pdf文件

In [1]:

import os
import sys
import win32api
import win32con
from win32comext.shell.shell import ShellExecuteEx
from win32comext.shell import shellcon
import win32process
import time
import win32event
import _thread
 
def experimentGuide(index):
    if(index == 1):
        exePath = r'C:\Program Files (x86)\Foxit Software\Foxit Reader\FoxitReader.exe'
        helplFile = r'C:\Users\Administrator\Desktop\资料\师傅带徒弟学：Python图形用户界面编程wxPython视频课程\wxPython实战(中文版)\wxPython实战(中文版).pdf'
 
        # handle = win32api.ShellExecuteEx(0, 'open', helplFile, '', '', 1)
        procInfo = ShellExecuteEx(nShow=win32con.SW_SHOWNORMAL,
                                  fMask=shellcon.SEE_MASK_NOCLOSEPROCESS,
                                  lpVerb='open',
                                 lpFile=helplFile,
                                  lpParameters='')
        # handle = win32process.CreateProcess(exePath, helplFile, None, None, 0, win32process.CREATE_NO_WINDOW, None, None, win32process.STARTUPINFO())
        print('wait for end')
        #等待进程结束
        print(win32event.WaitForSingleObject(procInfo['hProcess'], -1))
        time.sleep(5)
        print('close it')

if __name__ == '__main__':
    experimentGuide(1)


wait for end
0
close it


In [3]:
import win32con
from win32comext.shell.shell import ShellExecuteEx
from win32comext.shell import shellcon
import time
import win32event


class OpenPDF(object):
    def experimentGuide(self,index):
        if(index == 1):
            #exePath = r'C:\Program Files (x86)\Foxit Software\Foxit Reader\FoxitReader.exe'
            helplFile = r'C:\Users\Administrator\Desktop\资料\师傅带徒弟学：Python图形用户界面编程wxPython视频课程\wxPython实战(中文版)\wxPython实战(中文版).pdf'
     
            # handle = win32api.ShellExecuteEx(0, 'open', helplFile, '', '', 1)
            procInfo = ShellExecuteEx(nShow=win32con.SW_SHOWNORMAL,
                                      fMask=shellcon.SEE_MASK_NOCLOSEPROCESS,
                                      lpVerb='open',
                                      lpFile=helplFile,
                                      lpParameters='')
            # handle = win32process.CreateProcess(exePath, helplFile, None, None, 0, win32process.CREATE_NO_WINDOW, None, None, win32process.STARTUPINFO())
            print('wait for end')
            #等待进程结束
            print(win32event.WaitForSingleObject(procInfo['hProcess'], -1))
            time.sleep(5)
            print('close it')

if __name__ == '__main__':
    op = OpenPDF()
    op.experimentGuide(1)

wait for end
0
close it
